# Trial 1

In [ ]:
import requests
from lxml import html
from urllib.request import urlopen, Request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time
from datetime import datetime
import threading
from datetime import date
from apscheduler.schedulers.background import BackgroundScheduler


#launch url
url = "https://pure360.pure-yoga.com/zh-tw/TW?location_id=24"

# create a new Firefox session
browser = webdriver.Chrome()
browser.implicitly_wait(30)
browser.get(url)

sign_in_button = browser.find_element_by_id('sign-in-btn')
time.sleep(1)
sign_in_button.click()
time.sleep(2)
username = browser.find_element_by_id("username")
username.clear()
username.send_keys("huohsien@gmail.com")
time.sleep(1)
password = browser.find_element_by_id("password")
password.clear()
password.send_keys("jj121632")
time.sleep(1)
login_button = browser.find_element_by_name('login')
login_button.click()
time.sleep(1)
data_soup = BeautifulSoup(browser.page_source,'lxml')
result = data_soup.find_all(attrs={"data-class-id": True}, limit = 1)



def fun_click_reserve_button():
    global browser
    reserve_button = browser.find_element_by_xpath("//button[@data-class-id='9058']")
    time.sleep(1)
    reserve_button.click()
    time.sleep(1)
    #browser.refresh()
    #time.sleep(5)
    #fun_click_reserve_button()

# Define the function that is to be executed
def my_job(text):
    print(text)
    global browser
    browser.quit()

# Start the scheduler
sched = BackgroundScheduler()
job = sched.add_job(my_job, 'date', run_date=datetime(2019, 1, 4, 14, 38, 5), args=['text'])
sched.start()




    

# Trial 2 (better)

In [ ]:
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from datetime import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler


#launch url
url = "https://pure360.pure-yoga.com/en/TW?location_id=24"

# create a new Chrome session
browser = webdriver.Chrome()
browser.implicitly_wait(30)
browser.get(url)

sign_in_button = browser.find_element_by_id('sign-in-btn')
time.sleep(1)
sign_in_button.click()
time.sleep(2)
username = browser.find_element_by_id("username")
username.clear()
username.send_keys("huohsien@gmail.com")
time.sleep(1)
password = browser.find_element_by_id("password")
password.clear()
password.send_keys("jj121632")
time.sleep(1)
login_button = browser.find_element_by_name('login')
login_button.click()
time.sleep(1)
data_soup = BeautifulSoup(browser.page_source,'lxml')

### NOT WORKING

In [ ]:
tds = data_soup.findAll("td",attrs={"data-date":"Sat Jan 5","data-time":"08:30"})
for td in tds:
    if td.find('button'):
        print(type(button))


### Try another way and succeeded!

In [ ]:
def reserve_class(browser, str_date, str_time):
    # class to be booked, specify date and time
    td = browser.find_element_by_xpath("//td[@data-date='" + str_date + "'][@data-time='" + str_time + "']")
    button = td.find_element_by_xpath(".//button")
    button.click()

In [ ]:
# Start the scheduler
sched = BackgroundScheduler()
job = sched.add_job(reserve_class, 'date', run_date=datetime(2019, 1, 5, 0, 26, 5), args=[browser, 'Sat Jan 5', '08:30'])
sched.start()

# Pure Tower

In [ ]:
reserve_class(browser, "Mon Jan 7", "16:30")
reserve_class(browser, "Mon Jan 7", "17:45")
reserve_class(browser, "Mon Jan 7", "19:00") #
reserve_class(browser, "Tue Jan 8", "15:00")
reserve_class(browser, "Tue Jan 8", "16:30")
reserve_class(browser, "Tue Jan 8", "19:00")
reserve_class(browser, "Tue Jan 8", "20:15")

# complete Program

In [6]:
#imports

from lxml import html
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from datetime import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler
import sys

#settings
url = "https://pure360.pure-yoga.com/en/TW?location_id=24"
classes = [{"date":"Sun Jan 6","time":"08:30"},{"date":"Sun Jan 6","time":"12:15"}]

book_datetime = datetime(2019, 1, 6, 2, 5, 50)
is_running = True

# create a new Chrome session
browser = webdriver.Chrome()
browser.implicitly_wait(30)
browser.get(url)

sign_in_button = browser.find_element_by_id('sign-in-btn')
time.sleep(1)
sign_in_button.click()
time.sleep(2)
username = browser.find_element_by_id("username")
username.clear()
username.send_keys("huohsien@gmail.com")
time.sleep(1)
password = browser.find_element_by_id("password")
password.clear()
password.send_keys("jj121632")
time.sleep(1)
login_button = browser.find_element_by_name('login')
login_button.click()
time.sleep(1)
data_soup = BeautifulSoup(browser.page_source,'lxml')

def reserve_class(browser, classes):
    for c in classes:
       # class to be booked, specify date and time
        td = browser.find_element_by_xpath("//td[@data-date='" + c['date'] + "'][@data-time='" + c['time'] + "']")
        button = td.find_element_by_xpath(".//button")
        button.click()
        time.sleep(4)
    
    global is_running
    time.sleep(6)
    is_running = False
    browser.quit()


# Start the scheduleruler
scheduler = BackgroundScheduler()
job = scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[browser, classes])
scheduler.start()

while is_running:
    time.sleep(60)
    sys.stdout.write('.'); sys.stdout.flush()


.

# ------------------------

# scrape without Selenum

In [ ]:
session_requests = requests.session()
login_url = "https://pure360.pure-yoga.com/zh-tw/TW?location_id=24"
result = session_requests.get(login_url)

tree = html.fromstring(result.text)
authenticity_token = list(set(tree.xpath("//meta[@name='csrf-token']/@content")))[0]

payload = {
	"username": "huohsien@gmail.com", 
	"password": "jj121632"
}

result = session_requests.post(
	login_url, 
	data = payload, 
	headers = {"user-agent":"Mozilla/5.0 (Windows NT 6.1; rv:57.0) Gecko/20100101 Firefox/57.0"}
)

result

In [ ]:
def scrape(link):
    try:
        headers = {}
        headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
        return str(urlopen(Request(link, headers = headers)).read())
    except Exception as e:
        print(str(e))

In [ ]:
scrape("https://pure360.pure-yoga.com/zh-tw/TW?location_id=24")